In [ ]:
!pip install numpy pandas googlemaps matplotlib ortools

In [2]:
import pandas as pd

delivery_date = {
    'Customer': ['Mr. Ankit', 'Mrs. Sharma', 'Ms. Nikita', 'Dr. Goyal'],
    'Address': ['Sector-110', 'Palam Vihar', 'Sector-1', 'Ashok Vihar'],
    'Delivery_Time': [30, 25, 20, 15]
}

df = pd.DataFrame(delivery_date)
df

,Customer,Address,Delivery_Time
0,Mr. Ankit,Sector-110,30
1,Mrs. Sharma,Palam Vihar,25
2,Ms. Nikita,Sector-1,20
3,Dr. Goyal,Ashok Vihar,15


In [3]:
distance_matrix = [
  [0, 10, 15, 20],
  [10, 0, 35, 25],
  [15, 35, 0, 30],
  [20, 25, 30, 0]
]

distance_df = pd.DataFrame(distance_matrix, columns=df['Customer'], index=df['Customer'])
distance_df

Customer,Mr. Ankit,Mrs. Sharma,Ms. Nikita,Dr. Goyal
Customer,,,,
Mr. Ankit,0,10,15,20
Mrs. Sharma,10,0,35,25
Ms. Nikita,15,35,0,30
Dr. Goyal,20,25,30,0


In [4]:
!pip install --upgrade ortools

In [12]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model(distance_matrix):
  data = {}
  data['distance_matrix'] = distance_matrix
  data['num_vehicles'] = 1
  data['depot'] = 0
  return data

def solve_tsp(data):
  manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
  routing = pywrapcp.RoutingModel(manager)

  def distance_callback(from_index, to_index):
    return data['distance_matrix'][manager.IndexToNode(from_index)][manager.IndexToNode(to_index)]

  transit_callback_index = routing.RegisterTransitCallback(distance_callback)
  routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

  search_parameters = pywrapcp.DefaultRoutingSearchParameters()
  search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

  solution = routing.SolveWithParameters(search_parameters)

  if solution:
    route = []
    index = routing.Start(0)
    while not routing.IsEnd(index):
      route.append(manager.IndexToNode(index))
      index = solution.Value(routing.NextVar(index))
    return route
  else:
      return None


data = create_data_model(distance_matrix)
optimal_route = solve_tsp(data)
optimal_route

[0, 1, 3, 2]